# PCAP L2 — Exercise 2 (Answers Notebook)

`Mission 02`: Design custom exceptions, use `raise`, `raise from`, and `assert` as pre-flight safety checks.

**Context:** You're validating a thruster burn command. Inputs arrive as strings from ground control and must be validated and converted. You'll enforce rules (no negative fuel, temp within limits), chain low-level parsing errors to high-level “pilot-friendly” errors, and assert critical invariants.

> 🎯 Your Key Objectives:
> - Define custom exception classes with meaningful names.
> - Use `raise` for business-rule enforcement (not `assert`).
> - Use `raise ... from ...` to preserve the root causes.
> - Use `assert` only for internal developer assumptions (invariants). Never rely on it for user/security checks.

## Part 1: Define Mission Exceptions

Your Tasks:
- [ ] 1.1 — Create a base exception `FlightControlError(Exception)` with a short docstring.
- [ ] 1.2 — Create `BadCommandError(FlightControlError)` for malformed/invalid command strings.
- [ ] 1.3 — Create `ThrusterConfigError(FlightControlError)` for rule violations (fuel/duration/temp ranges).
- [ ] 1.4 — Add a simple `__init__` statement to store a message or raw input as context.

In [ ]:
# === Part 1: Your code here ===

# 1.1)
class FlightControlError(Exception):
    """Base for mission control errors."""
    pass

# 1.2)
class BadCommandError(FlightControlError):
    """Raised when a command string is malformed or incompatible."""
    # 1.4)
    def __init__(self, message=None, cause=None):
        self.cause = cause
        super().__init__(message)

# 1.3)
class ThrusterConfigError(FlightControlError):
    """Raised when thruster parameters violate mission rules."""
    # 1.4)
    def __init__(self, message=None):
        super().__init__(message)


# Part 2: Parse & Validate Burn Command (`raise` + `raise from`)

Command Format:
`"fuel=<int>;duration=<int>;temp=<float>"`

Flight Rules to Keep in Mind:
- `fuel` > 0 (integer)
- `duration` > 0 (integer)
- `temp` in range `-100.0` … `1800.0` (float)

Your Tasks:
- [ ] 2.1 — Implement `parse_burn_command(cmd: str) -> dict`.
- [ ] 2.2 — Split into key/value pairs; on structure errors, `raise BadCommandError(...) **from** original_error`.
- [ ] 2.3 — Convert to `int/int/float`; on type errors, `raise BadCommandError(...) **from** original_error`.
- [ ] 2.4 — Enforce rules using `raise ThrusterConfigError(...)` with clear messages.
- [ ] 2.5 — Return a dict `{"fuel": int, "duration": int, "temp": float}` on success.

In [ ]:
# === Part 2: Your code here ===

# 2.1)
def parse_burn_command(cmd: str) -> dict:
    """
    Parse a burn command string into validated values.
    Returns: {"fuel": int, "duration": int, "temp": float}
    Raises: BadCommandError, ThrusterConfigError
    """

    # 2.2)
    parts = {}
    try:
        # very explicit parsing: split by ';', then each token by '='
        tokens = [t.strip() for t in cmd.split(";") if t.strip()]
        for tok in tokens:
            if "=" not in tok:
                raise ValueError(f"missing '=' in token: {tok}")
            k, v = tok.split("=", 1)
            k, v = k.strip(), v.strip()
            if not k:
                raise ValueError(f"empty key in token: {tok}")
            parts[k] = v
    except Exception as e:
        # 2.2 — preserve root cause
        raise BadCommandError(f"Invalid structure: {cmd}", cause=e) from e

    # 2.3)
    try:
        fuel = int(parts.get("fuel", ""))
        duration = int(parts.get("duration", ""))
        temp = float(parts.get("temp", ""))
    except (TypeError, ValueError) as e:
        raise BadCommandError(f"Type conversion failed in '{cmd}'", cause=e) from e

    # 2.4)
    if fuel <= 0:
        raise ThrusterConfigError(f"Fuel must be positive (got {fuel})")
    if duration <= 0:
        raise ThrusterConfigError(f"Duration must be positive (got {duration})")
    if not (-100.0 <= temp <= 1800.0):
        raise ThrusterConfigError(f"Temp out of range (-100..1800): {temp}")

    # 2.5)
    return {"fuel": fuel, "duration": duration, "temp": temp}

## Part 3: Compute Burn Energy (Assertions for Invariants)

Context: Use `asser` for internal invariants only (they can be disabled with `-O`).

Your Tasks:
- [ ] 3.1 — Implement `compute_burn(params: dict) -> float`.
- [ ] 3.2 — Extract `fuel`, `duration`, and `temp` from params.
- [ ] 3.3 — Assert developer assumptions (fuel/duration already validated > 0).
- [ ] 3.4 — Compute a toy energy score:
`energy = fuel * duration * max(0.5, (1 + temp / 1000.0))`
- [ ] 3.5 — Assert `energy >= 0` and return `energy`.

In [ ]:
# === Part 3: Your code here ===

# 3.1)
def compute_burn(params: dict) -> float:
    # 3.2)
    fuel = params["fuel"]
    duration = params["duration"]
    temp = params["temp"]

    # 3.3) Internal invariants (developer assumptions)
    assert fuel > 0 and duration > 0, "fuel/duration should be positive post-validate"

    # 3.4) Compute energy
    energy = fuel * duration * max(0.5, (1 + temp / 1000.0))

    # 3.5)
    assert energy >= 0, "energy should not be negative"
    return energy


## Part 4: Mission Sim — Exercise All Paths (and View Chaining)

Context: Run the following without modification after implementing Parts 1-3. Observe messages and chained causes (look for `e.__cause__`).

In [ ]:
# === Part 4: Run (no edits) ===

tests = [
    "fuel=50;duration=10;temp=800.0",      # OK
    "fuel=0;duration=5;temp=300.0",        # ThrusterConfigError (fuel)
    "fuel=25;duration=-3;temp=300.0",      # ThrusterConfigError (duration)
    "fuel=10;duration=5;temp=2000.0",      # ThrusterConfigError (temp)
    "fuel=abc;duration=5;temp=100.0",      # BadCommandError (type)
    "duration=5;temp=500.0",               # BadCommandError (missing fuel -> type)
    "fuel=10;duration=5;temp=notanumber",  # BadCommandError (type)
]

for t in tests:
    print("\n---", t)
    try:
        p = parse_burn_command(t)
        e = compute_burn(p)
        print(f"ENERGY: {e:.2f}")
    except BadCommandError as e:
        print(f"[BadCommandError] {e}")
        if getattr(e, "cause", None):
            print(" ↳ caused by:", type(e.cause).__name__, e.cause)
    except ThrusterConfigError as e:
        print(f"[ThrusterConfigError] {e}")


--- fuel=50;duration=10;temp=800.0
ENERGY: 900.00

--- fuel=0;duration=5;temp=300.0
[ThrusterConfigError] Fuel must be positive (got 0)

--- fuel=25;duration=-3;temp=300.0
[ThrusterConfigError] Duration must be positive (got -3)

--- fuel=10;duration=5;temp=2000.0
[ThrusterConfigError] Temp out of range (-100..1800): 2000.0

--- fuel=abc;duration=5;temp=100.0
[BadCommandError] Type conversion failed in 'fuel=abc;duration=5;temp=100.0'
 ↳ caused by: ValueError invalid literal for int() with base 10: 'abc'

--- duration=5;temp=500.0
[BadCommandError] Type conversion failed in 'duration=5;temp=500.0'
 ↳ caused by: ValueError invalid literal for int() with base 10: ''

--- fuel=10;duration=5;temp=notanumber
[BadCommandError] Type conversion failed in 'fuel=10;duration=5;temp=notanumber'
 ↳ caused by: ValueError could not convert string to float: 'notanumber'


## Part 5: Debrief (short answers)

### Q1 of 3: Why use `raise ... from ...` when handling low-level errors during parsing?

*Your answers here:*

Because it preserves the original exception as the `__cause__`, keeping the full traceback and low-level details for debugging while surfacing a clear, higher-level message.
_______

### Q2 of 3: When should you prefer `raise` over `assert`?

*Your answers here:*

For user input validation, business rules, and security checks. `assert` is for developer invariants and can be disabled with `-O`.

_______

### Q3 of 3: Give one benefit of defining custom exception classes for a domain (e.g., flight control).

*Your answers here:*

They let you categorize and handle domain-specific failures precisely (clean API contract, richer context fields, and targeted recovery strategies).

_______